# 1.- Librerias



In [0]:
# cargo librerías necesarias
import numpy  as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt # para dibujar
%matplotlib inline

Ruta en google drive del proyecto

In [0]:
ruta = "/content/gdrive/My Drive/KeepCoding/LightGBM/"

# 2.- Carga de datos train y test

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Cargo los datos para train

In [4]:
path_csv_train = ruta + 'data/listings_train.csv'
house_madrid_train = pd.read_csv(path_csv_train, sep =',', decimal='.') # cargamos fichero
print(f"El dataset contiene {len(house_madrid_train)} viviendas Airbnb con {len(house_madrid_train.columns)} características")
pd.set_option('display.max_columns', len(house_madrid_train)) # Para ver todas las columnas
house_madrid_train.head(5)  

El dataset contiene 9060 viviendas Airbnb con 219 características


,Unnamed: 0,Price,id,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,number_of_reviews,instant_bookable,sentiment,Topic_general,Topic_descripcion,Topic_atracciones,Topic_servicios,Topic_transporte,Dias_activo,Luggage dropoff allowed,TV,Room-darkening shades,Paid parking on premises,Microwave,Smoking allowed,Laptop friendly workspace,Dryer,Host greets you,Dishwasher,Coffee maker,Oven,Cooking basics,Hair dryer,Washer,Stove,Patio or balcony,Elevator,Hot water,Fire extinguisher,Paid parking off premises,Family/kid friendly,Cable TV,Air conditioning,Private entrance,Pets allowed,Refrigerator,Extra pillows and blankets,Hangers,Wide entrance,Long term stays allowed,Internet,Smoke detector,Bed linens,Iron,Lock on bedroom door,Buzzer/wireless intercom,First aid kit,Free street parking,Breakfast,Crib,Shampoo,No stairs or steps to enter,host_response_time_a few days or more,host_response_time_unknown,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_100%,host_response_rate_50-89%,host_response_rate_90-99%,neighbourhood_cleansed_Acacias,neighbourhood_cleansed_Argüelles,neighbourhood_cleansed_Cortes,neighbourhood_cleansed_Cuatro Caminos,neighbourhood_cleansed_Delicias,neighbourhood_cleansed_Embajadores,neighbourhood_cleansed_Goya,neighbourhood_cleansed_Guindalera,neighbourhood_cleansed_Ibiza,neighbourhood_cleansed_Justicia,neighbourhood_cleansed_Numancia,neighbourhood_cleansed_Otro_Barrio,neighbourhood_cleansed_Pacífico,neighbourhood_cleansed_Palacio,neighbourhood_cleansed_Palos de Moguer,neighbourhood_cleansed_Puerta del Angel,neighbourhood_cleansed_Recoletos,neighbourhood_cleansed_San Diego,neighbourhood_cleansed_Sol,neighbourhood_cleansed_Trafalgar,neighbourhood_cleansed_Universidad,property_type_Apartment,property_type_Other,room_type_Entire home/apt,room_type_Hotel room,room_type_Shared room,review_scores_rating_0-49/100,review_scores_rating_50-79/100,review_scores_rating_80-94/100,review_scores_rating_95-100/100,review_scores_accuracy_0-4/10,review_scores_accuracy_10/10,review_scores_accuracy_5-7/10,review_scores_accuracy_8/10,review_scores_accuracy_9/10,review_scores_cleanliness_0-4/10,review_scores_cleanliness_10/10,review_scores_cleanliness_5-7/10,review_scores_cleanliness_8/10,review_scores_cleanliness_9/10,review_scores_checkin_0-4/10,review_scores_checkin_10/10,review_scores_checkin_5-7/10,review_scores_checkin_8/10,review_scores_checkin_9/10,review_scores_communication_0-4/10,review_scores_communication_10/10,review_scores_communication_5-7/10,review_scores_communication_8/10,review_scores_communication_9/10,review_scores_location_0-4/10,review_scores_location_10/10,review_scores_location_5-7/10,review_scores_location_8/10,review_scores_location_9/10,review_scores_value_0-4/10,review_scores_value_10/10,review_scores_value_5-7/10,review_scores_value_8/10,review_scores_value_9/10,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,dist_to_station_+5km,dist_to_station_0-200m,dist_to_station_1km-3km,dist_to_station_200m-500m,dist_to_station_3km-5km,dist_to_station_500m-1km,dist_to_parking_+5km,dist_to_parking_0-200m,dist_to_parking_1km-3km,dist_to_parking_200m-500m,dist_to_parking_3km-5km,dist_to_parking_500m-1km,dist_to_museo1_+5km,dist_to_museo1_0-200m,dist_to_museo1_1km-3km,dist_to_museo1_200m-500m,dist_to_museo1_3km-5km,dist_to_museo1_500m-1km,dist_to_museo2_+5km,dist_to_museo2_0-200m,dist_to_museo2_1km-3km,dist_to_museo2_200m-500m,dist_to_museo2_3km-5km,dist_to_museo2_500m-1km,dist_to_museo3_+5km,dist_to_museo3_0-200m,dist_to_museo3_1km-3km,dist_to_museo3_200m-500m,dist_to_museo3_3km-5km,dist_to_museo3_500m-1km,dist_to_museo4_+5km,dist_to_museo4_0-200m,dist_to_museo4_1km-3km,dist_to_museo4_200m-500m,dist_to_museo4_3km-5km,dist_to_museo4_500m-1km,dist_to_museo5_

Cargo los datos para test

In [5]:
path_csv_test = ruta + 'data/listings_test.csv'
house_madrid_test = pd.read_csv(path_csv_test, sep =',', decimal='.') # cargamos fichero
print(f"El dataset contiene {len(house_madrid_test)} viviendas Airbnb con {len(house_madrid_test.columns)} características")
pd.set_option('display.max_columns', len(house_madrid_test)) # Para ver todas las columnas
house_madrid_test.head(5) 

El dataset contiene 3872 viviendas Airbnb con 219 características


,Unnamed: 0,Price,id,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,number_of_reviews,instant_bookable,sentiment,Topic_general,Topic_descripcion,Topic_atracciones,Topic_servicios,Topic_transporte,Dias_activo,Luggage dropoff allowed,TV,Room-darkening shades,Paid parking on premises,Microwave,Smoking allowed,Laptop friendly workspace,Dryer,Host greets you,Dishwasher,Coffee maker,Oven,Cooking basics,Hair dryer,Washer,Stove,Patio or balcony,Elevator,Hot water,Fire extinguisher,Paid parking off premises,Family/kid friendly,Cable TV,Air conditioning,Private entrance,Pets allowed,Refrigerator,Extra pillows and blankets,Hangers,Wide entrance,Long term stays allowed,Internet,Smoke detector,Bed linens,Iron,Lock on bedroom door,Buzzer/wireless intercom,First aid kit,Free street parking,Breakfast,Crib,Shampoo,No stairs or steps to enter,host_response_time_a few days or more,host_response_time_unknown,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_100%,host_response_rate_50-89%,host_response_rate_90-99%,neighbourhood_cleansed_Acacias,neighbourhood_cleansed_Argüelles,neighbourhood_cleansed_Cortes,neighbourhood_cleansed_Cuatro Caminos,neighbourhood_cleansed_Delicias,neighbourhood_cleansed_Embajadores,neighbourhood_cleansed_Goya,neighbourhood_cleansed_Guindalera,neighbourhood_cleansed_Ibiza,neighbourhood_cleansed_Justicia,neighbourhood_cleansed_Numancia,neighbourhood_cleansed_Otro_Barrio,neighbourhood_cleansed_Pacífico,neighbourhood_cleansed_Palacio,neighbourhood_cleansed_Palos de Moguer,neighbourhood_cleansed_Puerta del Angel,neighbourhood_cleansed_Recoletos,neighbourhood_cleansed_San Diego,neighbourhood_cleansed_Sol,neighbourhood_cleansed_Trafalgar,neighbourhood_cleansed_Universidad,property_type_Apartment,property_type_Other,room_type_Entire home/apt,room_type_Hotel room,room_type_Shared room,review_scores_rating_0-49/100,review_scores_rating_50-79/100,review_scores_rating_80-94/100,review_scores_rating_95-100/100,review_scores_accuracy_0-4/10,review_scores_accuracy_10/10,review_scores_accuracy_5-7/10,review_scores_accuracy_8/10,review_scores_accuracy_9/10,review_scores_cleanliness_0-4/10,review_scores_cleanliness_10/10,review_scores_cleanliness_5-7/10,review_scores_cleanliness_8/10,review_scores_cleanliness_9/10,review_scores_checkin_0-4/10,review_scores_checkin_10/10,review_scores_checkin_5-7/10,review_scores_checkin_8/10,review_scores_checkin_9/10,review_scores_communication_0-4/10,review_scores_communication_10/10,review_scores_communication_5-7/10,review_scores_communication_8/10,review_scores_communication_9/10,review_scores_location_0-4/10,review_scores_location_10/10,review_scores_location_5-7/10,review_scores_location_8/10,review_scores_location_9/10,review_scores_value_0-4/10,review_scores_value_10/10,review_scores_value_5-7/10,review_scores_value_8/10,review_scores_value_9/10,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,dist_to_station_+5km,dist_to_station_0-200m,dist_to_station_1km-3km,dist_to_station_200m-500m,dist_to_station_3km-5km,dist_to_station_500m-1km,dist_to_parking_+5km,dist_to_parking_0-200m,dist_to_parking_1km-3km,dist_to_parking_200m-500m,dist_to_parking_3km-5km,dist_to_parking_500m-1km,dist_to_museo1_+5km,dist_to_museo1_0-200m,dist_to_museo1_1km-3km,dist_to_museo1_200m-500m,dist_to_museo1_3km-5km,dist_to_museo1_500m-1km,dist_to_museo2_+5km,dist_to_museo2_0-200m,dist_to_museo2_1km-3km,dist_to_museo2_200m-500m,dist_to_museo2_3km-5km,dist_to_museo2_500m-1km,dist_to_museo3_+5km,dist_to_museo3_0-200m,dist_to_museo3_1km-3km,dist_to_museo3_200m-500m,dist_to_museo3_3km-5km,dist_to_museo3_500m-1km,dist_to_museo4_+5km,dist_to_museo4_0-200m,dist_to_museo4_1km-3km,dist_to_museo4_200m-500m,dist_to_museo4_3km-5km,dist_to_museo4_500m-1km,dist_to_museo5_

Elimino las columnas Unname

In [0]:
house_madrid_train.drop(house_madrid_train.columns[house_madrid_train.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
house_madrid_test.drop(house_madrid_test.columns[house_madrid_test.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Elimino las columnas "id"

In [0]:
house_madrid_train.drop ('id', axis=1, inplace=True)
house_madrid_test.drop ('id', axis=1, inplace=True)

#3.- Separación X e Y. Normalización

Train

In [0]:
# Separamos X and y
X_train = house_madrid_train.drop('Price', axis=1)
y_train = house_madrid_train.Price

# Scaling
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=list(X_train.columns))

Test

In [0]:
# Separamos X and y
X_test = house_madrid_test.drop('Price', axis=1)
y_test = house_madrid_test.Price

# Scaling
scaler = StandardScaler()
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=list(X_test.columns)) 

## 4.- Modelado

Voy a utilizar el algoritmo LightGBM de Machine Learning

Al haber dividido en train y test antes de cualquier procesado y haber realizado la conversión de variables categóricas con el metodo dumy podría ocurrir que tenga diferente número de características en un cojunto y en otro. Voya asegurarme que tengo en los dos lo mismo y que las características están ordenadas de la misma manera:

In [0]:
missing_cols = set (X_train.columns) - set (X_test.columns)
#Caso de que que en test no tenga las mismas columnas las añado con valor 0
for c in missing_cols:
    X_test [c]=0
#Me aseguro que el orden de las columnas en test sea el mismo que en train
X_test = X_test[X_train.columns]

In [11]:
X_train.head(1)

,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,number_of_reviews,instant_bookable,sentiment,Topic_general,Topic_descripcion,Topic_atracciones,Topic_servicios,Topic_transporte,Dias_activo,Luggage dropoff allowed,TV,Room-darkening shades,Paid parking on premises,Microwave,Smoking allowed,Laptop friendly workspace,Dryer,Host greets you,Dishwasher,Coffee maker,Oven,Cooking basics,Hair dryer,Washer,Stove,Patio or balcony,Elevator,Hot water,Fire extinguisher,Paid parking off premises,Family/kid friendly,Cable TV,Air conditioning,Private entrance,Pets allowed,Refrigerator,Extra pillows and blankets,Hangers,Wide entrance,Long term stays allowed,Internet,Smoke detector,Bed linens,Iron,Lock on bedroom door,Buzzer/wireless intercom,First aid kit,Free street parking,Breakfast,Crib,Shampoo,No stairs or steps to enter,host_response_time_a few days or more,host_response_time_unknown,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_100%,host_response_rate_50-89%,host_response_rate_90-99%,neighbourhood_cleansed_Acacias,neighbourhood_cleansed_Argüelles,neighbourhood_cleansed_Cortes,neighbourhood_cleansed_Cuatro Caminos,neighbourhood_cleansed_Delicias,neighbourhood_cleansed_Embajadores,neighbourhood_cleansed_Goya,neighbourhood_cleansed_Guindalera,neighbourhood_cleansed_Ibiza,neighbourhood_cleansed_Justicia,neighbourhood_cleansed_Numancia,neighbourhood_cleansed_Otro_Barrio,neighbourhood_cleansed_Pacífico,neighbourhood_cleansed_Palacio,neighbourhood_cleansed_Palos de Moguer,neighbourhood_cleansed_Puerta del Angel,neighbourhood_cleansed_Recoletos,neighbourhood_cleansed_San Diego,neighbourhood_cleansed_Sol,neighbourhood_cleansed_Trafalgar,neighbourhood_cleansed_Universidad,property_type_Apartment,property_type_Other,room_type_Entire home/apt,room_type_Hotel room,room_type_Shared room,review_scores_rating_0-49/100,review_scores_rating_50-79/100,review_scores_rating_80-94/100,review_scores_rating_95-100/100,review_scores_accuracy_0-4/10,review_scores_accuracy_10/10,review_scores_accuracy_5-7/10,review_scores_accuracy_8/10,review_scores_accuracy_9/10,review_scores_cleanliness_0-4/10,review_scores_cleanliness_10/10,review_scores_cleanliness_5-7/10,review_scores_cleanliness_8/10,review_scores_cleanliness_9/10,review_scores_checkin_0-4/10,review_scores_checkin_10/10,review_scores_checkin_5-7/10,review_scores_checkin_8/10,review_scores_checkin_9/10,review_scores_communication_0-4/10,review_scores_communication_10/10,review_scores_communication_5-7/10,review_scores_communication_8/10,review_scores_communication_9/10,review_scores_location_0-4/10,review_scores_location_10/10,review_scores_location_5-7/10,review_scores_location_8/10,review_scores_location_9/10,review_scores_value_0-4/10,review_scores_value_10/10,review_scores_value_5-7/10,review_scores_value_8/10,review_scores_value_9/10,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,dist_to_station_+5km,dist_to_station_0-200m,dist_to_station_1km-3km,dist_to_station_200m-500m,dist_to_station_3km-5km,dist_to_station_500m-1km,dist_to_parking_+5km,dist_to_parking_0-200m,dist_to_parking_1km-3km,dist_to_parking_200m-500m,dist_to_parking_3km-5km,dist_to_parking_500m-1km,dist_to_museo1_+5km,dist_to_museo1_0-200m,dist_to_museo1_1km-3km,dist_to_museo1_200m-500m,dist_to_museo1_3km-5km,dist_to_museo1_500m-1km,dist_to_museo2_+5km,dist_to_museo2_0-200m,dist_to_museo2_1km-3km,dist_to_museo2_200m-500m,dist_to_museo2_3km-5km,dist_to_museo2_500m-1km,dist_to_museo3_+5km,dist_to_museo3_0-200m,dist_to_museo3_1km-3km,dist_to_museo3_200m-500m,dist_to_museo3_3km-5km,dist_to_museo3_500m-1km,dist_to_museo4_+5km,dist_to_museo4_0-200m,dist_to_museo4_1km-3km,dist_to_museo4_200m-500m,dist_to_museo4_3km-5km,dist_to_museo4_500m-1km,dist_to_museo5_+5km,dist_to_museo5_

In [12]:
X_test.head(1)

,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,number_of_reviews,instant_bookable,sentiment,Topic_general,Topic_descripcion,Topic_atracciones,Topic_servicios,Topic_transporte,Dias_activo,Luggage dropoff allowed,TV,Room-darkening shades,Paid parking on premises,Microwave,Smoking allowed,Laptop friendly workspace,Dryer,Host greets you,Dishwasher,Coffee maker,Oven,Cooking basics,Hair dryer,Washer,Stove,Patio or balcony,Elevator,Hot water,Fire extinguisher,Paid parking off premises,Family/kid friendly,Cable TV,Air conditioning,Private entrance,Pets allowed,Refrigerator,Extra pillows and blankets,Hangers,Wide entrance,Long term stays allowed,Internet,Smoke detector,Bed linens,Iron,Lock on bedroom door,Buzzer/wireless intercom,First aid kit,Free street parking,Breakfast,Crib,Shampoo,No stairs or steps to enter,host_response_time_a few days or more,host_response_time_unknown,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_100%,host_response_rate_50-89%,host_response_rate_90-99%,neighbourhood_cleansed_Acacias,neighbourhood_cleansed_Argüelles,neighbourhood_cleansed_Cortes,neighbourhood_cleansed_Cuatro Caminos,neighbourhood_cleansed_Delicias,neighbourhood_cleansed_Embajadores,neighbourhood_cleansed_Goya,neighbourhood_cleansed_Guindalera,neighbourhood_cleansed_Ibiza,neighbourhood_cleansed_Justicia,neighbourhood_cleansed_Numancia,neighbourhood_cleansed_Otro_Barrio,neighbourhood_cleansed_Pacífico,neighbourhood_cleansed_Palacio,neighbourhood_cleansed_Palos de Moguer,neighbourhood_cleansed_Puerta del Angel,neighbourhood_cleansed_Recoletos,neighbourhood_cleansed_San Diego,neighbourhood_cleansed_Sol,neighbourhood_cleansed_Trafalgar,neighbourhood_cleansed_Universidad,property_type_Apartment,property_type_Other,room_type_Entire home/apt,room_type_Hotel room,room_type_Shared room,review_scores_rating_0-49/100,review_scores_rating_50-79/100,review_scores_rating_80-94/100,review_scores_rating_95-100/100,review_scores_accuracy_0-4/10,review_scores_accuracy_10/10,review_scores_accuracy_5-7/10,review_scores_accuracy_8/10,review_scores_accuracy_9/10,review_scores_cleanliness_0-4/10,review_scores_cleanliness_10/10,review_scores_cleanliness_5-7/10,review_scores_cleanliness_8/10,review_scores_cleanliness_9/10,review_scores_checkin_0-4/10,review_scores_checkin_10/10,review_scores_checkin_5-7/10,review_scores_checkin_8/10,review_scores_checkin_9/10,review_scores_communication_0-4/10,review_scores_communication_10/10,review_scores_communication_5-7/10,review_scores_communication_8/10,review_scores_communication_9/10,review_scores_location_0-4/10,review_scores_location_10/10,review_scores_location_5-7/10,review_scores_location_8/10,review_scores_location_9/10,review_scores_value_0-4/10,review_scores_value_10/10,review_scores_value_5-7/10,review_scores_value_8/10,review_scores_value_9/10,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,dist_to_station_+5km,dist_to_station_0-200m,dist_to_station_1km-3km,dist_to_station_200m-500m,dist_to_station_3km-5km,dist_to_station_500m-1km,dist_to_parking_+5km,dist_to_parking_0-200m,dist_to_parking_1km-3km,dist_to_parking_200m-500m,dist_to_parking_3km-5km,dist_to_parking_500m-1km,dist_to_museo1_+5km,dist_to_museo1_0-200m,dist_to_museo1_1km-3km,dist_to_museo1_200m-500m,dist_to_museo1_3km-5km,dist_to_museo1_500m-1km,dist_to_museo2_+5km,dist_to_museo2_0-200m,dist_to_museo2_1km-3km,dist_to_museo2_200m-500m,dist_to_museo2_3km-5km,dist_to_museo2_500m-1km,dist_to_museo3_+5km,dist_to_museo3_0-200m,dist_to_museo3_1km-3km,dist_to_museo3_200m-500m,dist_to_museo3_3km-5km,dist_to_museo3_500m-1km,dist_to_museo4_+5km,dist_to_museo4_0-200m,dist_to_museo4_1km-3km,dist_to_museo4_200m-500m,dist_to_museo4_3km-5km,dist_to_museo4_500m-1km,dist_to_museo5_+5km,dist_to_museo5_

## 4.1.- Modelo LightGBM

In [0]:
gridParams = { 
    'max_depth': range(1,20),
    'num_leaves': [31,50],
    'num_trees': [300,400,500],
    'objective': ['regression'],
    
}

In [20]:
grid = GridSearchCV (lgb.LGBMRegressor(),param_grid=gridParams, cv=5)
grid.fit(X_train,y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(1, 20),
                         'max_depthobjective': ['regression'],
                         'num_leaves': [31, 50], 'num_trees': [30

In [21]:
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))


best mean cross-validation score: 0.779
best parameters: {'max_depth': 13, 'max_depthobjective': 'regression', 'num_leaves': 31, 'num_trees': 500}


In [0]:
num_leaves_optimo = grid.best_params_['num_leaves']
num_trees_optimo = grid.best_params_['num_trees']
objetive_optimo = 'regression'
max_depth_optimo = grid.best_params_['max_depth']

In [0]:
params = {'num_leaves': num_leaves_optimo, 
          'num_trees_optimo': num_trees_optimo,
          'objective': objetive_optimo,
          'max_depth': max_depth_optimo,
           }

In [0]:
#Modelado con valores óptimos
training_data = lgb.Dataset(data = X_train, label = y_train)

Model = lgb.train(params = params, train_set = training_data, num_boost_round = 10)

In [39]:
#Predicción

ytrainarbol = Model.predict(X_train)
ytestarbol = Model.predict(X_test)

mseTrainModelarbol = mean_squared_error(y_train,ytrainarbol)
mseTestModelarbol = mean_squared_error(y_test,ytestarbol)

r2TrainModelarbol = r2_score (y_train,ytrainarbol)
r2TestModelarbol = r2_score (y_test,ytestarbol)

print('RMSE Modelo  (train): %0.3g' % np.sqrt(mseTrainModelarbol))
print('RMSE Modelo  (test) : %0.3g' % np.sqrt(mseTestModelarbol))
print ('R^2 train = %0.3g' %r2TrainModelarbol)
print ('R^2 test =%0.3g' %r2TestModelarbol)



RMSE Modelo  (train): 0.373
RMSE Modelo  (test) : 0.376
R^2 train = 0.607
R^2 test =0.6
